In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# input data

In [ ]:
df=pd.read_csv('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
df.head().T

In [ ]:
df.shape

# Explore dataset

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
plt.figure()
df.hist(rwidth=0.9)
plt.tight_layout()

In [ ]:
sns.countplot(x=df['quality'])

# setting threshold for quality

good= >6
bad= <6

In [ ]:
df['quality'] = [1 if i > 6 else 0 for i in df['quality']]
sns.countplot(x=df['quality'])

In [ ]:
from pandas.plotting import scatter_matrix
attr=list(df.columns)

In [ ]:
scatter_matrix(df[attr],figsize=(200,150))
plt.tight_layout()

In [ ]:
df.corr()

In [ ]:
#df=df.drop(['residual sugar','free sulfur dioxide','pH'],axis=1)

In [ ]:
df.isnull().sum()

In [ ]:
matrix = np.triu(df.corr())
sns.heatmap(df.corr(), annot = True, fmt='.1g',vmin=-1, vmax=1, center= 0, cmap= 'coolwarm',mask=matrix)

# splitting dataset

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop(['quality'], axis=1)
y = df['quality']

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=.20, random_state=42)

# feature scaling

Feature scaling is a method used to normalize the range of independent variables or features of data.

In [ ]:
# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train = norm.transform(X_train)

# transform testing dataabs
X_test = norm.transform(X_test)

# KNN classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier()


# Hyper parameter tuning

In [ ]:
# For Hyper-parameter Tuning the model
from sklearn.model_selection import GridSearchCV

# For checking Model Performance
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import learning_curve

import warnings
warnings.simplefilter(action="ignore")

from sklearn.model_selection import StratifiedKFold

In [ ]:

cv_method = StratifiedKFold(n_splits=3)
param={'n_neighbors':(1,3,5,7),'weights': ('uniform','distance'),'metric':('euclidean','manhattan','chebyshev','minkowski','wminkowski','seuclidean','mahalanobis'),'p' :(1,2)}
clf = GridSearchCV(neigh, param,cv=cv_method,scoring="accuracy")
clf.fit(X_train,Y_train)

In [ ]:
clf.best_params_

In [ ]:
knn = KNeighborsClassifier(n_neighbors= 5, p= 1, weights='distance',metric= 'euclidean')
knn.fit(X_train,Y_train)
knn.score(X_test,Y_test)

# cheking performance measures

In [ ]:
pred = knn.predict(X_test)
print("Classification Report")
print(classification_report(Y_test, pred))
ax= plt.subplot()
sns.heatmap(confusion_matrix(Y_test, pred), annot=True, ax = ax);

# labels, title and ticks
ax.set_xlabel("Predicted labels")
ax.set_ylabel("True labels")
ax.set_title("Confusion Matrix")
ax.xaxis.set_ticklabels(["bad", "good"])